# Bit Commitment for trusting parties
Bit Commitment (BC) is a cryptographic primitive that captures the following two-party functionality: Alice has a bit b that she wants to commit to Bob, but she wants to prevent Bob from reading b until she chooses to reveal it (concealing or hiding). Although Bob should not be able to determine b before Alice reveals it, Alice should be unable to change the bit after it is committed (binding). A physical-world implementation of bit commitment would be for Alice to write b on a piece of paper, lock it in a safe, and send the safe to Bob. Since Bob cannot open the safe, he cannot determine b (concealing), and since Alice has physically given the safe to Bob, she cannot change b after the commitment phase (binding). When Alice wishes to reveal the bit, she sends the key to Bob.

## Importing libraries

In [4]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib

## Funtions to convert strings to and from binary

In [5]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

In [ ]:
def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

## Sender side (Alice)

In [23]:
# Converting the string into binary
messageToSend = tobits("Hello, this is Yajat Malhotra!!")
messageToSend

'01001000011001010110110001101100011011110010110000100000011101000110100001101001011100110010000001101001011100110010000001011001011000010110101001100001011101000010000001001101011000010110110001101000011011110111010001110010011000010010000100100001'

In [24]:
# An array for random 1s/0s of the same length as the message
Alice_bases = [randint(0,1) for x in range(len(messageToSend))]

In [26]:
# Creating a quantum circuit
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(messageToSend[i] == "1"):
            mycircuit.x(0)
    else:
        if(messageToSend[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

## Receiver's Side (Bob)

In [27]:
backend = Aer.get_backend('qasm_simulator')
measurements = list()
for i in range(len(Alice_bases)):
    qubit = qubits[i]
    if(Alice_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))
print("Alice message was: " + frombits(measurements))

Alice message was: Hello, this is Yajat Malhotra!!
